In [27]:

from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import sqlite3

def reiksmes_filtras(driver, selektorius) :
    try :
        return driver.find_element(By.CSS_SELECTOR, selektorius).text
    except :
        return ""
    
with sqlite3.connect("lemora.db") as prisijungimas:
    c = prisijungimas.cursor()
    # c.execute("DROP TABLE prekes")
    c.execute("CREATE TABLE IF NOT EXISTS prekes (id INTEGER PRIMARY KEY AUTOINCREMENT, pavadinimas TEXT, kaina TEXT, prekes_zenklas TEXT, kodas TEXT)")

    nustatymai = Options()

    # Nustatymo pridėjimas, jog naršyklė bus atidaroma fone
    # nustatymai.add_argument('--headless')

    driver = webdriver.Chrome(options=nustatymai)

    pagrindinis_url = "https://lemora.lt/576-cementas/"

    # Uzklausos adresas
    driver.get(pagrindinis_url)

    driver.maximize_window()

    sleep(.5)

    # # Sutikimas su sausainėlių politika
    mygtukas = driver.find_element(By.CSS_SELECTOR, "#CybotCookiebotDialogBodyButtonAccept")

    mygtukas.click()

    # # produktas = driver.find_element(By.CSS_SELECTOR, ".product-grid__item:last-child")
    # # produktas.find_element(By.CSS_SELECTOR, ".card__url").click()

    urls = []

    while True :
        sleep(1)

        produktai = driver.find_elements(By.CSS_SELECTOR, ".product_img_link")

        for produktas in produktai :
            urls.append(produktas.get_attribute("href"))

        try :
            driver.find_element(By.CSS_SELECTOR, ".pagination_next a").click()  
        except Exception as error :
            # print(error)
            print("Paskutinis puslapis pasiektas")
            break

    # print(len(urls))

    # Adresų apdorojimas

    for url in urls :
        driver.get(url)

        pavadinimas = reiksmes_filtras(driver, ".product-main-right h1")

        kaina = reiksmes_filtras(driver, "#our_price_display")

        prekes_zenklas = reiksmes_filtras(driver, "#product_manufacturer a")

        kodas = reiksmes_filtras(driver, "#product_reference span")

        print(pavadinimas, kaina, prekes_zenklas, kodas)

        c.execute(f"SELECT COUNT(*) FROM prekes WHERE kodas = '{kodas}'")

        if c.fetchone()[0] == 0 :
            # PREKĖS ĮRAŠYMAS Į DUOMENŲ BAZĘ
            c.execute(f"INSERT INTO prekes (pavadinimas, kaina, prekes_zenklas, kodas) VALUES ('{pavadinimas}', '{kaina}', '{prekes_zenklas}', '{kodas}')")
            



Paskutinis puslapis pasiektas
Cementas su superplastikliais ir mikrofibra Rocket M-800, 35 kg 7,44 € HEIDELBERGCEMENT A000102166
Cementas ROCKET FIBRO, su superplastikliais, 35kg 7,89 € HEIDELBERGCEMENT A000100229
Cementas montažinis Ceresit CX 5 EXPRESS, greitai kietėjantis, 5-25kg 9,45 € CERESIT A000004709
Cementas Eco Standard CEM II/B-M (LL-P) 42,5 N, 25kg 5,12 € AKMENĖS CEMENTAS A000119944


In [29]:
with sqlite3.connect("lemora.db") as prisijungimas:
    c = prisijungimas.cursor()
    prekes = c.execute("SELECT * FROM prekes")

    for preke in prekes :
        print(preke)


(1, 'Cementas su superplastikliais ir mikrofibra Rocket M-800, 35 kg', '7,44 €', 'HEIDELBERGCEMENT', 'A000102166')
(2, 'Cementas ROCKET FIBRO, su superplastikliais, 35kg', '7,89 €', 'HEIDELBERGCEMENT', 'A000100229')
(3, 'Cementas montažinis Ceresit CX 5 EXPRESS, greitai kietėjantis, 5-25kg', '9,45 €', 'CERESIT', 'A000004709')
(4, 'Cementas Eco Standard CEM II/B-M (LL-P) 42,5 N, 25kg', '5,12 €', 'AKMENĖS CEMENTAS', 'A000119944')
(5, 'Cementas su superplastikliais ir mikrofibra Rocket M-800, 35 kg', '7,44 €', 'HEIDELBERGCEMENT', 'A000102166')
(6, 'Cementas ROCKET FIBRO, su superplastikliais, 35kg', '7,89 €', 'HEIDELBERGCEMENT', 'A000100229')
(7, 'Cementas montažinis Ceresit CX 5 EXPRESS, greitai kietėjantis, 5-25kg', '9,45 €', 'CERESIT', 'A000004709')
(8, 'Cementas Eco Standard CEM II/B-M (LL-P) 42,5 N, 25kg', '5,12 €', 'AKMENĖS CEMENTAS', 'A000119944')


In [28]:
with sqlite3.connect("lemora.db") as prisijungimas:
    c = prisijungimas.cursor()
    c.execute("SELECT COUNT(*) FROM prekes WHERE kodas = 'A000100229'")

    # Prekių kiekio susigrąžinimas
    print(c.fetchone()[0])

2


In [51]:

from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import sqlite3

def reiksmes_filtras(driver, selektorius) :
    try :
        return driver.find_element(By.CSS_SELECTOR, selektorius).text
    except :
        return ""
    
login = "magebit"
password = "Demo123"
    
with sqlite3.connect("parduotuve.db") as prisijungimas:
    c = prisijungimas.cursor()
    c.execute("DROP TABLE IF EXISTS prekes")
    c.execute("CREATE TABLE IF NOT EXISTS prekes (id INTEGER PRIMARY KEY AUTOINCREMENT, pavadinimas TEXT, kodas TEXT, kaina INTEGER, kiekis INTEGER)")

    nustatymai = Options()

    # Nustatymo pridėjimas, jog naršyklė bus atidaroma fone
    # nustatymai.add_argument('--headless')

    driver = webdriver.Chrome(options=nustatymai)

    pagrindinis_url = "https://magento2-demo.magebit.com/admin"

    # Uzklausos adresas
    driver.get(pagrindinis_url)

    driver.maximize_window()

    # sleep(.5)

    # PRISIJUNGIMO VARDO LAUKELIS
    driver.find_element(By.CSS_SELECTOR, "#username").send_keys(login)
    # SLAPTAŽODŽIO LAUKELIS
    driver.find_element(By.CSS_SELECTOR, "#login").send_keys(password)

    # MYGTUKO PASPAUDIMAS
    driver.find_element(By.CSS_SELECTOR, ".action-primary").click()

    driver.find_element(By.CSS_SELECTOR, "[data-ui-id=\"menu-magento-catalog-catalog\"] a").click()

    sleep(.5)
    
    driver.find_element(By.CSS_SELECTOR, "[data-ui-id=\"menu-magento-catalog-catalog-products\"] a").click()

    sleep(3)

    produktai = driver.find_elements(By.CSS_SELECTOR, "[data-role=\"grid\"] tbody tr")

    for produktas in produktai :
        stulpeliai = produktas.find_elements(By.CSS_SELECTOR, "td")

        pavadinimas = stulpeliai[3].text.replace("'", "")
        kodas = stulpeliai[6].text
        
        kaina = stulpeliai[7].text
        kaina = kaina.replace("$", "") if kaina != "" else 0

        kiekis = stulpeliai[8].text

        print(pavadinimas, kodas, kaina, kiekis)

        c.execute(f"INSERT INTO prekes (pavadinimas, kodas, kaina, kiekis) VALUES ('{pavadinimas}', '{kodas}', '{kaina}', '{kiekis}')")

    # # Adresų apdorojimas

    # for url in urls :
    #     driver.get(url)

    #     pavadinimas = reiksmes_filtras(driver, ".product-main-right h1")

    #     kaina = reiksmes_filtras(driver, "#our_price_display")

    #     prekes_zenklas = reiksmes_filtras(driver, "#product_manufacturer a")

    #     kodas = reiksmes_filtras(driver, "#product_reference span")

    #     print(pavadinimas, kaina, prekes_zenklas, kodas)

    #     c.execute(f"SELECT COUNT(*) FROM prekes WHERE kodas = '{kodas}'")

    #     if c.fetchone()[0] == 0 :
    #         # PREKĖS ĮRAŠYMAS Į DUOMENŲ BAZĘ
    #         c.execute(f"INSERT INTO prekes (pavadinimas, kaina, prekes_zenklas, kodas) VALUES ('{pavadinimas}', '{kaina}', '{prekes_zenklas}', '{kodas}')")
            



Joust Duffle Bag 24-MB01 34.00 100.0000
Strive Shoulder Pack 24-MB04 32.00 100.0000
Crown Summit Backpack 24-MB03 38.00 100.0000
Wayfarer Messenger Bag 24-MB05 0 0.0000
Rival Field Messenger 24-MB06 45.00 100.0000
Fusion Backpack 24-MB02 59.00 100.0000
Impulse Duffle 24-UB02 74.00 100.0000
Voyage Yoga Bag 24-WB01 32.00 100.0000
Compete Track Tote 24-WB02 32.00 100.0000
Savvy Shoulder Tote 24-WB05 32.00 100.0000
Endeavor Daytrip Backpack 24-WB06 33.00 100.0000
Driven Backpack 24-WB03 36.00 100.0000
Overnight Duffle 24-WB07 45.00 100.0000
Push It Messenger Bag 24-WB04 45.00 100.0000
Affirm Water Bottle 24-UG06 7.00 100.0000
Dual Handle Cardio Ball 24-UG07 12.00 100.0000
Zing Jump Rope 24-UG04 12.00 100.0000
Pursuit Lumaflex™ Tone Band 24-UG02 16.00 100.0000
Go-Getr Pushup Grips 24-UG05 19.00 100.0000
Quest Lumaflex™ Band 24-UG01 19.00 100.0000
Sprite Foam Yoga Brick 24-WG084 5.00 100.0000
Sprite Foam Roller 24-WG088 19.00 100.0000
Harmony Lumaflex™ Strength Band Kit 24-UG03 22.00 100.000

In [52]:
with sqlite3.connect("parduotuve.db") as prisijungimas:
    c = prisijungimas.cursor()
    prekes = c.execute("SELECT * FROM prekes")

    for preke in prekes :
        print(preke)


(1, 'Joust Duffle Bag', '24-MB01', 34, 100)
(2, 'Strive Shoulder Pack', '24-MB04', 32, 100)
(3, 'Crown Summit Backpack', '24-MB03', 38, 100)
(4, 'Wayfarer Messenger Bag', '24-MB05', 0, 0)
(5, 'Rival Field Messenger', '24-MB06', 45, 100)
(6, 'Fusion Backpack', '24-MB02', 59, 100)
(7, 'Impulse Duffle', '24-UB02', 74, 100)
(8, 'Voyage Yoga Bag', '24-WB01', 32, 100)
(9, 'Compete Track Tote', '24-WB02', 32, 100)
(10, 'Savvy Shoulder Tote', '24-WB05', 32, 100)
(11, 'Endeavor Daytrip Backpack', '24-WB06', 33, 100)
(12, 'Driven Backpack', '24-WB03', 36, 100)
(13, 'Overnight Duffle', '24-WB07', 45, 100)
(14, 'Push It Messenger Bag', '24-WB04', 45, 100)
(15, 'Affirm Water Bottle', '24-UG06', 7, 100)
(16, 'Dual Handle Cardio Ball', '24-UG07', 12, 100)
(17, 'Zing Jump Rope', '24-UG04', 12, 100)
(18, 'Pursuit Lumaflex™ Tone Band', '24-UG02', 16, 100)
(19, 'Go-Getr Pushup Grips', '24-UG05', 19, 100)
(20, 'Quest Lumaflex™ Band', '24-UG01', 19, 100)
(21, 'Sprite Foam Yoga Brick', '24-WG084', 5, 100)
(